In [6]:
#Import all the neccessary libraries
from keras.models import Sequential
from keras.layers import Conv2D, ZeroPadding2D, Activation, Input, concatenate
from keras.models import Model
from keras.layers.normalization import BatchNormalization
from keras.layers.pooling import MaxPooling2D, AveragePooling2D
from keras.layers.merge import Concatenate
from keras.layers.core import Lambda, Flatten, Dense
from keras.initializers import glorot_uniform
from keras.engine.topology import Layer
from keras import backend as K
K.set_image_data_format('channels_first')
import cv2
import os
import numpy as np
from numpy import genfromtxt
import pandas as pd
import tensorflow as tf
from fr_utils import *
from inception_blocks_v2 import *

%matplotlib inline
%load_ext autoreload
%autoreload 2

# np.set_printoptions(threshold=np.nan)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
#Load the weights already trained, here we load FaceNet model
FRmodel = faceRecoModel(input_shape=(3, 96, 96))
print("Total Params:", FRmodel.count_params())

Total Params: 3743280


In [10]:
#  This model ensures that the output is an encoding vector of size 128. Here we need encoding that can perform two simple task i.e. 
#  encoding of two images of same person must be close while the encoding of images of two differnt people must vary a lot.
# To serve this purpose we define triplet loss function.
def triplet_loss(y_true, y_pred, alpha = 0.2):
        
    anchor, positive, negative = y_pred[0], y_pred[1], y_pred[2] 
    #  Compute the (encoding) distance between the anchor and the positive 
    #  Compute the (encoding) distance between the anchor and the negative, you will need to sum over axis=-1
    #  Subtract the two previous distances and add alpha.
    # Take the maximum of basic_loss and 0.0. Sum over the training examples.
    pos_dist = tf.reduce_sum(tf.square(anchor-positive),axis=-1)
    neg_dist = tf.reduce_sum(tf.square(anchor-negative),axis=-1)
    basic_loss = pos_dist-neg_dist+alpha
    loss = tf.reduce_sum(tf.maximum(basic_loss,0.0))
    
    return loss

In [11]:
with tf.Session() as test:
    tf.set_random_seed(1)
    y_true = (None, None, None)
    y_pred = (tf.random_normal([3, 128], mean=6, stddev=0.1, seed = 1),
              tf.random_normal([3, 128], mean=1, stddev=1, seed = 1),
              tf.random_normal([3, 128], mean=3, stddev=4, seed = 1))
    loss = triplet_loss(y_true, y_pred)
    
    print("Loss = " + str(loss.eval()))

Loss = 528.1432


In [13]:
##FaceNet is trained by minimizing triplet loss, but here we laod weights from a previously trained model. This may take time.
FRmodel.compile(optimizer = 'adam', loss = triplet_loss, metrics = ['accuracy'])
load_weights_from_FaceNet(FRmodel)

In [17]:
#Applying the model on the our dataset for this we store the encodigs in a dataset, to generate encodings we use the function 
#which generates the encoding by running forward propagation of the model on the specified image
database={}
database["tanmay"]=img_to_encoding("images/tanmay.jpg",FRmodel)
# database["utsav"]=img_to_encoding("images/utsav.jpeg",FRmodel)
database["himanshu"]=img_to_encoding("images/himanshu.png",FRmodel)
database["mummy"]=img_to_encoding("images/mummy.jpg",FRmodel)

In [29]:
# Now the verify function which computes the encoding from the image path provided and then computes the distance between the 
# encoding and the identity image stored in database and if this distance is less than 0.7 then the person is the same else no.
def verify(image_path, identity, database, model):    
    #  Compute the encoding for the image. 
    #  Compute distance with identity's image. 
    encoding = img_to_encoding(image_path,model) 
    dist = np.linalg.norm(encoding-database[identity])
    
    if dist<0.75:
        print("It's " + str(identity) + ", Adios!")
        ans = True
    else:
        print("It's not " + str(identity) + ", sorry")
        ans = False
    return dist, ans

In [30]:
#Checking on our data
verify("images/verif.jpg", "tanmay", database, FRmodel)

It's tanmay, Adios!


(0.73613393, True)

In [31]:
verify("images/verif.jpg", "mummy", database, FRmodel)

It's not mummy, sorry


(0.93025035, False)

In [32]:
# Face recognition
# The earlier function was to verify whether the name and image match toeach other on the other hand this function will be used to 
# take path to image as input and then find which person matches to the given image and if no image matches it it returns that
# the image is not in the database

def who_is_it(image_path, database, model):
    ##  Compute the target "encoding" for the image. 
    ##  Find the closest encoding ##
    # Initialize "min_dist" to a large value, say 100 
    encoding = img_to_encoding(image_path,model)
    min_dist = 100
    # Loop over the database dictionary's names and encodings.
    for (name, db_enc) in database.items():
        # Compute L2 distance between the target "encoding" and the current "emb" from the database. 
        dist = np.linalg.norm(encoding-db_enc)
        # If this distance is less than the min_dist, then set min_dist to dist, and identity to name.
        if dist<min_dist:
            min_dist = dist
            identity = name
    if min_dist > 0.75:
        print("Not in the database.")
    else:
        print ("it's " + str(identity) + ", the distance is " + str(min_dist))
    return min_dist, identity

In [33]:
#implementing the new face recognitin function
who_is_it("images/verif.jpg", database, FRmodel)

it's tanmay, the distance is 0.73613393


(0.73613393, 'tanmay')

In [34]:
# So here I have implemented a face recognition system using the concept of transfer learning where we use the pretrained weights 
# in the FaceNet model and then check whether this works upon our dataset.